# 🧩 1.7 – Laboratorio: Reto Global de Expresiones Regulares

Aplicarás todas las técnicas vistas:
- Búsqueda y coincidencias (`findall`)
- Grupos y cuantificadores
- Validaciones (`match`, `fullmatch`)
- Reemplazos (`sub`, funciones dinámicas`)

---
## 🎯 Objetivo
Procesar un registro de logs simulados con **usuarios, emails, fechas y errores**, limpiarlos y generar un resumen estructurado.

Resultados esperados:
- Lista de usuarios válidos
- Fechas normalizadas
- Emails anonimizados
- Conteo de errores

---
## 🧰 Dataset inicial

```text
[2025-10-31 09:10] user=ana EMAIL=ana@gmail.com ERROR=0
[2025-10-31 09:15] user=luis EMAIL=luis@hotmail.com ERROR=1
[2025/10/31 09:20] user=MARÍA EMAIL=invalid_email ERROR=0
[2025-10-31 09:25] user=pepe EMAIL=pepe@empresa.es ERROR=1
[2025-10-31 09:40] user=SOFIA EMAIL=sofia@data.es ERROR=2
[2025-10-31 10:00] user=juan EMAIL=juan@@example.com ERROR=1
```

In [ ]:
logs = '''[2025-10-31 09:10] user=ana EMAIL=ana@gmail.com ERROR=0\n
[2025-10-31 09:15] user=luis EMAIL=luis@hotmail.com ERROR=1\n
[2025/10/31 09:20] user=MARÍA EMAIL=invalid_email ERROR=0\n
[2025-10-31 09:25] user=pepe EMAIL=pepe@empresa.es ERROR=1\n
[2025-10-31 09:40] user=SOFIA EMAIL=sofia@data.es ERROR=2\n
[2025-10-31 10:00] user=juan EMAIL=juan@@example.com  ERROR=1'''

In [3]:
logs

'[2025-10-31 09:10] user=ana EMAIL=ana@gmail.com ERROR=0\n\n[2025-10-31 09:15] user=luis EMAIL=luis@hotmail.com ERROR=1\n\n[2025/10/31 09:20] user=MARÍA EMAIL=invalid_email ERROR=0\n\n[2025-10-31 09:25] user=pepe EMAIL=pepe@empresa.es ERROR=1\n\n[2025-10-31 09:40] user=SOFIA EMAIL=sofia@data.es ERROR=2\n\n[2025-10-31 10:00] user=juan EMAIL=juan@@example.com  ERROR=1'

## 1️⃣ Limpieza inicial

1. Sustituye `/` por `-` en las fechas.
2. Convierte todo el texto a minúsculas.
3. Elimina espacios repetidos.

👉 Usa `re.sub()` y valida con `assert`.

In [4]:
import re

In [ ]:
# Copia para no sobreescribir
copia_logs = logs
logs_limpio = re.sub(r'/', '-', copia_logs)         # sustituir / por - en las fechas


'[2025-10-31 09:10] user=ana EMAIL=ana@gmail.com ERROR=0\n\n[2025-10-31 09:15] user=luis EMAIL=luis@hotmail.com ERROR=1\n\n[2025-10-31 09:20] user=MARÍA EMAIL=invalid_email ERROR=0\n\n[2025-10-31 09:25] user=pepe EMAIL=pepe@empresa.es ERROR=1\n\n[2025-10-31 09:40] user=SOFIA EMAIL=sofia@data.es ERROR=2\n\n[2025-10-31 10:00] user=juan EMAIL=juan@@example.com  ERROR=1'

***Solución propuesta***

In [23]:
import re

# TODO: Aplica las tres transformaciones sobre logs
texto = logs  # copia de trabajo
# texto = ...
texto = re.sub(r'/','-',texto)
texto = texto.lower()
texto = re.sub(r'\s+', ' ',texto)
print(texto[:120])

[2025-10-31 09:10] user=ana email=ana@gmail.com error=0 [2025-10-31 09:15] user=luis email=luis@hotmail.com error=1 [202


In [24]:
# Tests de limpieza
assert '/' not in texto, '❌ Aún quedan fechas con /'
assert texto == texto.lower(), '❌ No está en minúsculas'
assert '  ' not in texto, '❌ Hay dobles espacios'
print('✅ Limpieza inicial correcta.')

✅ Limpieza inicial correcta.


💡 **Tip:**  
- `re.sub(r'/', '-', texto)` reemplaza las barras.
- `texto.lower()` convierte todo a minúsculas.
- `re.sub(r'\s+', ' ', texto)` elimina espacios múltiples.

✅ **Solución explicada:** tres transformaciones encadenadas para homogeneizar el texto antes de procesar.

## 2️⃣ Extracción de campos con grupos

Crea una expresión regular con grupos nombrados:
- `fecha`
- `user`
- `email`
- `error`

💡 Usa la sintaxis `(?P<nombre>...)`.

In [41]:
# TODO: completa el patrón
patron = r'(?P<nombre>\d{4})'
coincidencias = [m.groupdict() for m in re.finditer(patron, texto)]
print(coincidencias)
print(coincidencias[:3])

[{'nombre': '2025'}, {'nombre': '2025'}, {'nombre': '2025'}, {'nombre': '2025'}, {'nombre': '2025'}, {'nombre': '2025'}]
[{'nombre': '2025'}, {'nombre': '2025'}, {'nombre': '2025'}]


In [1]:
# Test extracción
assert len(coincidencias) == 6, f"❌ Se esperaban 6 coincidencias y hay {len(coincidencias)}"
for c in coincidencias:
    assert set(c.keys()) == {'fecha','user','email','error'}, '❌ Faltan campos en los grupos'
print('✅ Extracción correcta.')

NameError: name 'coincidencias' is not defined

💡 **Tip:**  `\[(?P<fecha>\d{4}-\d{2}-\d{2})[^]]*\] user=(?P<user>\w+) email=(?P<email>[\w@\.]+) error=(?P<error>\d+)`

✅ **Solución explicada:** los grupos capturan cada campo usando nombres y cuantificadores precisos.

## 3️⃣ Validación de emails

Filtra solo los registros con emails válidos usando:
```regex
^[\w\.-]+@[\w\.-]+\.\w+$
```

In [ ]:
# TODO: Filtra los registros válidos
patron_email = r''
validos = []

print(f'Emails válidos: {len(validos)} / {len(coincidencias)}')

In [ ]:
# Test emails válidos
assert len(validos) == 4, f"❌ Se esperaban 4 emails válidos y hay {len(validos)}"
assert all('email' in v for v in validos), '❌ Falta campo email'
print('✅ Validación de emails correcta.')

💡 **Tip:**  `validos = [r for r in coincidencias if re.fullmatch(patron_email, r['email'])]`

✅ **Solución explicada:** se valida con `re.fullmatch`, garantizando coincidencia exacta en todo el string.

## 4️⃣ Anonimización de emails

Reemplaza el nombre antes del `@` por `[anon]`.

Ejemplo: `ana@gmail.com` → `[anon]@gmail.com`

In [ ]:
# TODO: función de reemplazo
def anonimizar_email(match):
    pass

texto_anon = texto  # TODO: usa re.sub con función
print(texto_anon.split('\n')[:3])

In [ ]:
# Test anonimización
assert '[anon]@' in texto_anon, '❌ No se anonimizaron los emails'
print('✅ Anonimización correcta.')

💡 **Tip:**  `re.sub(r'([\w\.-]+)@([\w\.-]+)', lambda m: '[anon]@' + m.group(2), texto)`

✅ **Solución explicada:** `re.sub` aplica una función que devuelve `[anon]@` seguido del dominio original.

## 5️⃣ Conteo de errores por usuario

Genera un diccionario `{usuario: total_errores}` usando `findall()` y acumulación con `defaultdict`.

In [ ]:
from collections import defaultdict

# TODO: encuentra (user, error)
errores = defaultdict(int)
for user, num in []:
    errores[user] += int(num)

print(dict(errores))

In [ ]:
# Test conteo
assert len(errores) == 6, f"❌ Se esperaban 6 usuarios y hay {len(errores)}"
assert errores['sofia'] == 2, '❌ Error en conteo para sofia'
print('✅ Conteo correcto.')

💡 **Tip:**  `re.findall(r'user=(\w+)\s+email=[^ ]+\s+error=(\d+)', texto)`

✅ **Solución explicada:** se extraen pares `(usuario, número)` y se acumulan con `defaultdict(int)`.

## 6️⃣ Resultado final

Combina los datos válidos para mostrar:

| Usuario | Email | Errores |

In [ ]:
# TODO: recorre validos y muestra tabla
for r in validos:
    pass

💡 **Tip:**  usa el diccionario `errores[user]` para completar el conteo.

✅ **Solución explicada:** cada fila contiene el usuario, su email anonimizado y el número total de errores.

| Usuario | Email | Errores |
|:--------|:------------------|:---------|
| ana | [anon]@gmail.com | 0 |
| luis | [anon]@hotmail.com | 1 |
| pepe | [anon]@empresa.es | 1 |
| sofia | [anon]@data.es | 2 |

## 🧠 Reto adicional (opcional)

1. Valida fechas reales con `datetime.strptime`.
2. Exporta el resultado a CSV.
3. Implementa `procesar_logs(texto)` que devuelva el resumen final.

✅ **Solución explicada:** modulariza las fases y permite reutilizar la lógica con distintos datasets.

✅ **Fin del laboratorio – Expresiones Regulares**